In [1]:
import ray
import random
import time
import math
from fractions import Fraction


In [2]:
# Let's start Ray
ray.init(address='auto')

2022-12-13 09:37:20,402	INFO worker.py:1230 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
2022-12-13 09:37:20,800	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 10.0.63.8:9031...
2022-12-13 09:37:20,850	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_buwxbm99nq8dryqg6p8sbytw/services?redirect_to=dashboard 
2022-12-13 09:37:20,855	INFO packaging.py:373 -- Pushing file package 'gcs://_ray_pkg_217563e4f3ff2f6389ba2ad23d1e7cdc.zip' (0.11MiB) to Ray cluster...
2022-12-13 09:37:20,857	INFO packaging.py:386 -- Successfully pushed file package 'gcs://_ray_pkg_217563e4f3ff2f6389ba2ad23d1e7cdc.zip'.


Python version:,3.9.12
Ray version:,2.2.0
Dashboard:,http://console.anyscale.com/api/v2/sessions/ses_buwxbm99nq8dryqg6p8sbytw/services?redirect_to=dashboard


In [3]:
@ray.remote
def pi4_sample(sample_count):
    """pi4_sample runs sample_count experiments, and returns the 
    fraction of time it was inside the circle. 
    """
    in_count = 0
    for i in range(sample_count):
        x = random.random()
        y = random.random()
        if x*x + y*y <= 1:
            in_count += 1
    return Fraction(in_count, sample_count)

In [4]:
SAMPLE_COUNT = 1000 * 1000
start = time.time() 
future = pi4_sample.remote(sample_count = SAMPLE_COUNT)
pi4 = ray.get(future)
end = time.time()
dur = end - start
print(f'Running {SAMPLE_COUNT} tests took {dur} seconds')

Running 1000000 tests took 1.4959771633148193 seconds


In [6]:
pi = pi4 * 4
print(f'Pi={float(pi)} with error {abs(pi-math.pi)/pi}')

Pi=3.141364 with error 7.278799584934852e-05


# Let’s do 100 billion!

In [9]:
FULL_SAMPLE_COUNT = 100 * 1000 * 1000 * 1000 # 100 billion samples! 
BATCHES = int(FULL_SAMPLE_COUNT / SAMPLE_COUNT)
print(f'Doing {BATCHES} batches')
start = time.time() 
results = []
for _ in range(BATCHES):
    results.append(pi4_sample.remote(sample_count = SAMPLE_COUNT))
output = ray.get(results)
end = time.time()
dur = end - start
print(f'Running {BATCHES} batches of {SAMPLE_COUNT} tests took {dur} seconds')

Doing 100000 batches
(scheduler +5m43s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +5m43s) Adding 5 node(s) of type worker-node-type-0.
(scheduler +7m44s) Resized to 88 CPUs.
(scheduler +8m4s) Adding 5 node(s) of type worker-node-type-0.
(scheduler +9m34s) Resized to 152 CPUs.
(scheduler +10m29s) Resized to 168 CPUs.
Running 100000 batches of 1000000 tests took 543.4034125804901 seconds


In [11]:
pi = pi = sum(output)*4/len(output)
print(f'Pi={float(pi)} with error {abs(pi-math.pi)/pi}')

Pi=3.14159161972 with error 3.290910845453022e-07
